# Learning rate decay

Una cosa que se puede hacer para mejorar el entrenamiento es ir disminuyendo el valor del learning rate a medida que entrenamos. Veamos por qué

Si vemos el efecto de entrenar con un valor de learning rate de 0.01 en el ejemplo anterior

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

x = np.array( [ 0.        ,  0.34482759,  0.68965517,  1.03448276,  1.37931034,
        1.72413793,  2.06896552,  2.4137931 ,  2.75862069,  3.10344828,
        3.44827586,  3.79310345,  4.13793103,  4.48275862,  4.82758621,
        5.17241379,  5.51724138,  5.86206897,  6.20689655,  6.55172414,
        6.89655172,  7.24137931,  7.5862069 ,  7.93103448,  8.27586207,
        8.62068966,  8.96551724,  9.31034483,  9.65517241, 10.        ])

y = np.array( [-0.16281253,  1.88707606,  0.39649312,  0.03857752,  4.0148778 ,
        0.58866234,  3.35711859,  1.94314906,  6.96106424,  5.89792585,
        8.47226615,  3.67698542, 12.05958678,  9.85234481,  9.82181679,
        6.07652248, 14.17536744, 12.67825433, 12.97499286, 11.76098542,
       12.7843083 , 16.42241036, 13.67913705, 15.55066478, 17.45979602,
       16.41982806, 17.01977617, 20.28151197, 19.38148414, 19.41029831])

random.seed(45)
a = random.random()

def loss_fn(y, z):
    n = len(y)
    loss = np.sum((z-y) ** 2) / n
    return loss

posibles_a = np.linspace(0, 4, 300)
perdidas = np.empty_like(posibles_a)

for i in range (len(posibles_a)):
    z = posibles_a[i]*x
    perdidas[i] = loss_fn(y, z)

def gradiente (a, x, y):
    # Función que calcula el valor de una derivada en un punto
    n = len(y)
    return 2*np.sum((a*x - y)*x)/n

def gradiente_linea (i, a=None, error=None, gradiente=None, posibles_w=None, 
    losses=None, gradientes=None):
    # Función que devuleve los puntos de la linea que supone la derivada de una 
    # función en un punto dado
    if a is None:
        x1 = posibles_w[i]-0.7
        x2 = posibles_w[i]
        x3 = posibles_w[i]+0.7

        b = losses[i] - gradientes[i]*posibles_w[i]

        y1 = gradientes[i]*x1 + b
        y2 = losses[i]
        y3 = gradientes[i]*x3 + b
    else:
        x1 = a-0.7
        x2 = a
        x3 = a+0.7

        b = error - gradiente*a

        y1 = gradiente*x1 + b
        y2 = error
        y3 = gradiente*x3 + b

    x_linea = np.array([x1, x2, x3])
    y_linea = np.array([y1, y2, y3])

    return x_linea, y_linea

LRs = [2.5e-2]    # Tasa de aprendizaje o learning rate
steps = 50  # Numero de veces que se realiza el bucle de enrtenamiento

# Matrices donde se guardarán los datos para luego ver la evolución del entrenamiento en una gráfica
Zs = np.empty([len(LRs), steps, len(x)])
Xs_linea_gradiente = np.empty([len(LRs), steps, 3])
Ys_linea_gradiente = np.empty([len(LRs), steps, 3])
As = np.empty([len(LRs), steps])
Errores = np.empty([len(LRs), steps])

for l, lr in enumerate(LRs):
    # Inicialización aleatoria de a
    random.seed(45)
    a = random.random()
    
    print(f"Entrenamiento para lr = {lr}")
    for i in range(steps):
        # Calculamos el gradiente
        dl = gradiente(a, x, y)

        # Corregimos el valor de a
        a = a - lr*dl

        # Calculamos los valores que obtiene la red neuronal
        z = a*x

        # Obtenemos el error
        error = loss_fn(y, z)

        # Obtenemos las rectas de los gradientes para representarlas
        x_linea_gradiente, y_linea_gradiente = gradiente_linea(3, a=a, error=error, gradiente=dl)

        # Guardamos los valores para luego ver la evolución del entrenamiento en una gráfica
        As[l][i] = a
        Zs[l][i] = z
        Errores[l][i] = error
        Xs_linea_gradiente[l][i] = x_linea_gradiente
        Ys_linea_gradiente[l][i] = y_linea_gradiente

        # Imprimimos la evolución del entrenamiento
        if (i+1)%10 == 0:
            print(f"i={i+1}: error={error}, gradiente={dl}, a={a}")


# Creamos GIF con la evolución del entrenamiento
rectas = []
gradientes = []
puntos = []
a_texts = []
error_texts = []
lr_texts = []

fontsize = 12

# Creamos la gráfica inicial
fig, ax = plt.subplots(len(LRs),2, figsize=(15, 5*len(LRs)))
ax = ax.reshape(len(LRs),ax.shape[0])
fig.set_tight_layout(True)
for i in range(len(LRs)):
    ax[i][0].set_xlabel('X')
    ax[i][0].set_ylabel('Y  ', rotation=0)
    ax[i][1].set_xlabel('a')
    ax[i][1].set_ylabel('loss  ', rotation=0)
    ax[i][0].set_xlim(-0.5, 10.5)
    ax[i][0].set_ylim(-0.5, 20.5)
    ax[i][1].set_xlim(-0.5, 4.5)
    ax[i][1].set_ylim(-0.5, 140.5)

    # Se dibujan los datos que persistiran en toda la evolución de la gráfica
    ax[i][0].scatter(x, y)
    ax[i][1].plot(posibles_a, perdidas, linewidth = 3)

    # Se dibuja el resto de lineas que irán cambiando durante el entrenamiento
    line1, = ax[i][0].plot(x, Zs[i][0], 'k', linewidth=2)                             # Recta generada con la pendiente a aprendida
    line2, = ax[i][1].plot(Xs_linea_gradiente[i][0], Ys_linea_gradiente[i][0], 'g')   # Gradiente de la función de error
    punto, = ax[i][1].plot(As[i][0], Errores[i][0], 'r*')                             # Punto donde se calcula el gradiente
    rectas.append(line1)
    gradientes.append(line2)
    puntos.append(punto)

    # Se dibujan textos dentro de la segunda figura del subplot
    lr_text = ax[i][1].text(1, 100, f'lr = {LRs[i]}', fontsize = fontsize)
    a_text = ax[i][1].text(1, 90, f'pesos = {As[i][0]:.5f}', fontsize = fontsize)
    error_text = ax[i][1].text(1, 80, f'loss = {Errores[i][0]:.5f}', fontsize = fontsize)
    a_texts.append(a_text)
    error_texts.append(error_text)
    lr_texts.append(lr_text)
    
# Se dibuja un título
titulo = fig.suptitle(f'step: {0}', fontsize=fontsize)

# Se define la función que va a modificar la gráfica con la evolución del entrenamiento
def update(i):

    for l, _ in enumerate(LRs):
        # Se actualiza la recta generada con la pendiente a aprendida
        rectas[l].set_ydata(Zs[l][i])

        # Se actualiza el gradiente de la función de error
        gradientes[l].set_xdata(Xs_linea_gradiente[l][i])
        gradientes[l].set_ydata(Ys_linea_gradiente[l][i])

        # Se actualiza el punto 2. Punto donde se calcula el gradiente
        puntos[l].set_xdata(As[l][i])
        puntos[l].set_ydata(Errores[l][i])

        # Se actualizan los textos
        a_texts[l].set_text(f'pesos = {As[l][i]:.5f}')
        error_texts[l].set_text(f'loss = {Errores[l][i]:.5f}')
        lr_texts[l].set_text(f'lr = {LRs[l]}')
    
    titulo.set_text(f'step: {i}')

    return line1, # ax1, #line2, punto2, ax2, a_text, error_text

# Se crea la animación con un refresco cada 200 ms
interval = 500 # ms
anim = FuncAnimation(fig, update, frames=np.arange(0, steps), interval=interval)

# Se guarda en un GIF
gif_name = "GIFs/LR_noDecay.gif"
anim.save(gif_name, dpi=80, writer='imagemagick')
plt.close()

MovieWriter imagemagick unavailable; using Pillow instead.


Entrenamiento para lr = 0.025
i=10: error=3.5332415076132806, gradiente=4.434751401390874, a=1.945937013155588
i=20: error=3.463169558318404, gradiente=0.11728176690111386, a=1.990209371273261
i=30: error=3.46312055035669, gradiente=0.0031016423701155796, a=1.9913802013638249
i=40: error=3.463120516080773, gradiente=8.202626584997337e-05, a=1.991411165223878
i=50: error=3.4631205160568, gradiente=2.1692727570984022e-06, a=1.9914119840964342


![LR sin decaimiento](GIFs/LR_noDecay.gif)

Como se puede ver desde el step 4 el error no cambia mucho, en 4 steps se ha llegado a la mejor solución posible, pero vamos a ver si hacemos zoom en la zona del mínimo error

In [2]:
# Creamos GIF con la evolución del entrenamiento
rectas = []
gradientes1 = []
gradientes2 = []
puntos1 = []
puntos2 = []
a_texts1 = []
a_texts2 = []
error_texts1 = []
error_texts2 = []
lr_texts1 = []
lr_texts2 = []


fontsize = 12

# Creamos la gráfica inicial
fig, ax = plt.subplots(len(LRs),3, figsize=(15, 5*len(LRs)))
ax = ax.reshape(len(LRs),ax.shape[0])
fig.set_tight_layout(True)
for i in range(len(LRs)):
    ax[i][0].set_xlabel('X')
    ax[i][0].set_ylabel('Y  ', rotation=0)
    ax[i][1].set_xlabel('a')
    ax[i][1].set_ylabel('loss  ', rotation=0)
    ax[i][2].set_xlabel('a')
    ax[i][2].set_ylabel('loss  ', rotation=0)
    ax[i][0].set_xlim(-0.5, 10.5)
    ax[i][0].set_ylim(-0.5, 20.5)
    ax[i][1].set_xlim(-0.5, 4.5)
    ax[i][1].set_ylim(-0.5, 140.5)
    ax[i][2].set_xlim(1.8, 2.2)
    ax[i][2].set_ylim(3, 5.0)

    # Se dibujan los datos que persistiran en toda la evolución de la gráfica
    ax[i][0].scatter(x, y)
    ax[i][1].plot(posibles_a, perdidas, linewidth = 3)
    ax[i][2].plot(posibles_a, perdidas, linewidth = 3)

    # Se dibuja el resto de lineas que irán cambiando durante el entrenamiento
    recta, = ax[i][0].plot(x, Zs[i][0], 'k', linewidth=2)                             # Recta generada con la pendiente a aprendida
    gradiente1, = ax[i][1].plot(Xs_linea_gradiente[i][0], Ys_linea_gradiente[i][0], 'g')   # Gradiente de la función de error
    gradiente2, = ax[i][2].plot(Xs_linea_gradiente[i][0], Ys_linea_gradiente[i][0], 'g')   # Gradiente de la función de error
    punto1, = ax[i][1].plot(As[i][0], Errores[i][0], 'r*')                             # Punto donde se calcula el gradiente
    punto2, = ax[i][2].plot(As[i][0], Errores[i][0], 'r*')                             # Punto donde se calcula el gradiente
    rectas.append(recta)
    gradientes1.append(gradiente1)
    gradientes2.append(gradiente2)
    puntos1.append(punto1)
    puntos2.append(punto2)

    # Se dibujan textos dentro de la segunda figura del subplot
    lr_text1 = ax[i][1].text(1, 100, f'lr = {LRs[i]}', fontsize = fontsize)
    lr_text2 = ax[i][2].text(1.9, 4.75, f'lr = {LRs[i]}', fontsize = fontsize)
    a_text1 = ax[i][1].text(1, 90, f'pesos = {As[i][0]:.5f}', fontsize = fontsize)
    a_text2 = ax[i][2].text(1.9, 4.65, f'pesos = {As[i][0]:.5f}', fontsize = fontsize)
    error_text1 = ax[i][1].text(1, 80, f'loss = {Errores[i][0]:.5f}', fontsize = fontsize)
    error_text2 = ax[i][2].text(1.9, 4.55, f'loss = {Errores[i][0]:.5f}', fontsize = fontsize)
    a_texts1.append(a_text1)
    a_texts2.append(a_text2)
    error_texts1.append(error_text1)
    error_texts2.append(error_text2)
    lr_texts1.append(lr_text1)
    lr_texts2.append(lr_text2)
    
# Se dibuja un título
titulo = fig.suptitle(f'step: {0}', fontsize=fontsize)

# Se define la función que va a modificar la gráfica con la evolución del entrenamiento
def update(i):

    for l, _ in enumerate(LRs):
        # Se actualiza la recta generada con la pendiente a aprendida
        rectas[l].set_ydata(Zs[l][i])

        # Se actualiza el gradiente de la función de error
        gradientes1[l].set_xdata(Xs_linea_gradiente[l][i])
        gradientes1[l].set_ydata(Ys_linea_gradiente[l][i])
        gradientes2[l].set_xdata(Xs_linea_gradiente[l][i])
        gradientes2[l].set_ydata(Ys_linea_gradiente[l][i])

        # Se actualiza el punto 2. Punto donde se calcula el gradiente
        puntos1[l].set_xdata(As[l][i])
        puntos1[l].set_ydata(Errores[l][i])
        puntos2[l].set_xdata(As[l][i])
        puntos2[l].set_ydata(Errores[l][i])

        # Se actualizan los textos
        a_texts1[l].set_text(f'pesos = {As[l][i]:.5f}')
        a_texts2[l].set_text(f'pesos = {As[l][i]:.5f}')
        error_texts1[l].set_text(f'loss = {Errores[l][i]:.5f}')
        error_texts2[l].set_text(f'loss = {Errores[l][i]:.5f}')
        lr_texts1[l].set_text(f'lr = {LRs[l]}')
        lr_texts2[l].set_text(f'lr = {LRs[l]}')
    
    titulo.set_text(f'step: {i}')

    return line1, # ax1, #line2, punto2, ax2, a_text, error_text

# Se crea la animación con un refresco cada 200 ms
interval = 500 # ms
anim = FuncAnimation(fig, update, frames=np.arange(0, steps), interval=interval)

# Se guarda en un GIF
gif_name = "GIFs/LR_noDecay_zoom.gif"
anim.save(gif_name, dpi=80, writer='imagemagick')
plt.close()

MovieWriter imagemagick unavailable; using Pillow instead.


![LR sin decaimiento](GIFs/LR_noDecay_zoom.gif)

Como se puede ver, si se hace zoom, el punto va dando saltos al rededor del mínimo

Vemos qué pasa si vamos disminuyendo el valor del learning rate a medida que vamos entrenando

In [4]:
LRs = [2.5e-2, 2.5e-2]    # Tasa de aprendizaje o learning rate
steps = 20  # Numero de veces que se realiza el bucle de enrtenamiento

# Matrices donde se guardarán los datos para luego ver la evolución del entrenamiento en una gráfica
Zs = np.empty([len(LRs), steps, len(x)])
Xs_linea_gradiente = np.empty([len(LRs), steps, 3])
Ys_linea_gradiente = np.empty([len(LRs), steps, 3])
As = np.empty([len(LRs), steps])
Errores = np.empty([len(LRs), steps])
LR_values = np.empty([len(LRs), steps])
lr_decay = 0

for l, lr in enumerate(LRs):
    # Inicialización aleatoria de a
    random.seed(45)
    a = random.random()
    
    print(f"Entrenamiento para lr = {lr}")
    for i in range(steps):
        # Calculamos el gradiente
        dl = gradiente(a, x, y)

        # Corregimos el valor de a
        if l == 0:
            lr_decay = lr
            a -= lr * dl
        else:
            if i >= 0 and i < 5:
                lr_decay = lr
            elif i >= 5 and i < 10:
                lr_decay = lr/2
            elif i >= 10 and i < 15:
                lr_decay = lr/3
            elif i >= 15 and i < 20:
                lr_decay = lr/4
            a = a - lr_decay*dl

        # Calculamos los valores que obtiene la red neuronal
        z = a*x

        # Obtenemos el error
        error = loss_fn(y, z)

        # Obtenemos las rectas de los gradientes para representarlas
        x_linea_gradiente, y_linea_gradiente = gradiente_linea(3, a=a, error=error, gradiente=dl)

        # Guardamos los valores para luego ver la evolución del entrenamiento en una gráfica
        As[l][i] = a
        Zs[l][i] = z
        Errores[l][i] = error
        Xs_linea_gradiente[l][i] = x_linea_gradiente
        Ys_linea_gradiente[l][i] = y_linea_gradiente
        LR_values[l][i] = lr_decay

        # Imprimimos la evolución del entrenamiento
        if (i+1)%10 == 0:
            print(f"i={i+1}: error={error}, gradiente={dl}, a={a}")
        
    print()



# Creamos GIF con la evolución del entrenamiento
rectas = []
gradientes1 = []
gradientes2 = []
puntos1 = []
puntos2 = []
a_texts1 = []
a_texts2 = []
error_texts1 = []
error_texts2 = []
lr_texts1 = []
lr_texts2 = []


fontsize = 12

# Creamos la gráfica inicial
fig, ax = plt.subplots(len(LRs),3, figsize=(15, 5*len(LRs)))
fig.set_tight_layout(True)
for i in range(len(LRs)):
    ax[i][0].set_xlabel('X')
    ax[i][0].set_ylabel('Y  ', rotation=0)
    ax[i][1].set_xlabel('a')
    ax[i][1].set_ylabel('loss  ', rotation=0)
    ax[i][2].set_xlabel('a')
    ax[i][2].set_ylabel('loss  ', rotation=0)
    ax[i][0].set_xlim(-0.5, 10.5)
    ax[i][0].set_ylim(-0.5, 20.5)
    ax[i][1].set_xlim(-0.5, 4.5)
    ax[i][1].set_ylim(-0.5, 140.5)
    ax[i][2].set_xlim(1.8, 2.2)
    ax[i][2].set_ylim(3, 5.0)

    # Se dibujan los datos que persistiran en toda la evolución de la gráfica
    ax[i][0].scatter(x, y)
    ax[i][1].plot(posibles_a, perdidas, linewidth = 3)
    ax[i][2].plot(posibles_a, perdidas, linewidth = 3)

    # Se dibuja el resto de lineas que irán cambiando durante el entrenamiento
    recta, = ax[i][0].plot(x, Zs[i][0], 'k', linewidth=2)                             # Recta generada con la pendiente a aprendida
    gradiente1, = ax[i][1].plot(Xs_linea_gradiente[i][0], Ys_linea_gradiente[i][0], 'g')   # Gradiente de la función de error
    gradiente2, = ax[i][2].plot(Xs_linea_gradiente[i][0], Ys_linea_gradiente[i][0], 'g')   # Gradiente de la función de error
    punto1, = ax[i][1].plot(As[i][0], Errores[i][0], 'r*')                             # Punto donde se calcula el gradiente
    punto2, = ax[i][2].plot(As[i][0], Errores[i][0], 'r*')                             # Punto donde se calcula el gradiente
    rectas.append(recta)
    gradientes1.append(gradiente1)
    gradientes2.append(gradiente2)
    puntos1.append(punto1)
    puntos2.append(punto2)

    # Se dibujan textos dentro de la segunda figura del subplot
    lr_text1 = ax[i][1].text(1, 100, f'lr = {LR_values[i][0]:.5f}', fontsize = fontsize)
    lr_text2 = ax[i][2].text(1.9, 4.75, f'lr = {LR_values[i][0]:.5f}', fontsize = fontsize)
    a_text1 = ax[i][1].text(1, 90, f'pesos = {As[i][0]:.5f}', fontsize = fontsize)
    a_text2 = ax[i][2].text(1.9, 4.65, f'pesos = {As[i][0]:.5f}', fontsize = fontsize)
    error_text1 = ax[i][1].text(1, 80, f'loss = {Errores[i][0]:.5f}', fontsize = fontsize)
    error_text2 = ax[i][2].text(1.9, 4.55, f'loss = {Errores[i][0]:.5f}', fontsize = fontsize)
    a_texts1.append(a_text1)
    a_texts2.append(a_text2)
    error_texts1.append(error_text1)
    error_texts2.append(error_text2)
    lr_texts1.append(lr_text1)
    lr_texts2.append(lr_text2)
    
# Se dibuja un título
titulo = fig.suptitle(f'step: {0}', fontsize=fontsize)

# Se define la función que va a modificar la gráfica con la evolución del entrenamiento
def update(i):

    for l, _ in enumerate(LRs):
        # Se actualiza la recta generada con la pendiente a aprendida
        rectas[l].set_ydata(Zs[l][i])

        # Se actualiza el gradiente de la función de error
        gradientes1[l].set_xdata(Xs_linea_gradiente[l][i])
        gradientes1[l].set_ydata(Ys_linea_gradiente[l][i])
        gradientes2[l].set_xdata(Xs_linea_gradiente[l][i])
        gradientes2[l].set_ydata(Ys_linea_gradiente[l][i])

        # Se actualiza el punto 2. Punto donde se calcula el gradiente
        puntos1[l].set_xdata(As[l][i])
        puntos1[l].set_ydata(Errores[l][i])
        puntos2[l].set_xdata(As[l][i])
        puntos2[l].set_ydata(Errores[l][i])

        # Se actualizan los textos
        a_texts1[l].set_text(f'pesos = {As[l][i]:.5f}')
        a_texts2[l].set_text(f'pesos = {As[l][i]:.5f}')
        error_texts1[l].set_text(f'loss = {Errores[l][i]:.5f}')
        error_texts2[l].set_text(f'loss = {Errores[l][i]:.5f}')
        lr_texts1[l].set_text(f'lr = {LR_values[l][i]:.5f}')
        lr_texts2[l].set_text(f'lr = {LR_values[l][i]:.5f}')
    
    titulo.set_text(f'step: {i}')

    return line1, # ax1, #line2, punto2, ax2, a_text, error_text

# Se crea la animación con un refresco cada 200 ms
interval = 500 # ms
anim = FuncAnimation(fig, update, frames=np.arange(0, steps), interval=interval)

# Se guarda en un GIF
gif_name = "GIFs/LR_decay.gif"
anim.save(gif_name, dpi=80, writer='imagemagick')
plt.close()

MovieWriter imagemagick unavailable; using Pillow instead.


Entrenamiento para lr = 0.025
i=10: error=3.5332415076132806, gradiente=4.434751401390874, a=1.945937013155588
i=20: error=3.463169558318404, gradiente=0.11728176690111386, a=1.990209371273261

Entrenamiento para lr = 0.025
i=10: error=3.46312053385814, gradiente=0.010202610187635781, a=1.9914349189821918
i=20: error=3.4631205160568017, gradiente=2.662718301434571e-06, a=1.9914120289624948



![LR sin decaimiento](GIFs/LR_decay.gif)

Como se puede ver, cuando estamos cerca del mínimo error, es buena idea ir disminuyendo el valor del learning rate para que se ajuste mejor al problema.

Aquí se ha mostrado un ejemplo en el que se reduce el valor del learnig rate cada 5 steps, para que sea ilustrativo. Pero en la realidad, lo que se suele hacer es que si despues de varias épocas, el error, o la métrica que se esté usando, no mejora, se reduce el valor del learning rate